In [1]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import rdkit

import numpy as np 
print(rdkit.__version__)

2022.03.5


In [2]:
import os 

ms = []
names = []

for dir in os.listdir('./data'):
    if os.path.isdir('./data/'+dir):
        for file in os.listdir('./data/'+dir):
            if file.endswith('.pdb'):
                names.append(dir)
                m = rdkit.Chem.rdmolfiles.MolFromPDBFile('./data/'+dir+"/"+file)
                if m is None:
                    print("Error reading file:", file)
                else:
                    ms.append(m)
                    #print(type(m))


Error reading file: struc.pdb


[09:44:11] WARNING: not removing hydrogen atom without neighbors
[09:44:11] Explicit valence for atom # 0 Cl, 2, is greater than permitted


In [3]:

from rdkit import DataStructs

print("We have " , len(ms), " molecules ")

for fingersize in [8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]:

    mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=fingersize)
    rdgen = rdFingerprintGenerator.GetRDKitFPGenerator(fpSize=fingersize)
    apgen = rdFingerprintGenerator.GetAtomPairGenerator(fpSize=fingersize)
    ttgen = rdFingerprintGenerator.GetTopologicalTorsionGenerator(fpSize=fingersize)

    mf_maxnonzero = 0
    rd_maxnonzero = 0
    ap_maxnonzero = 0
    tt_maxnonzero = 0
    for m in ms:
        mf_np_bits = mfgen.GetFingerprintAsNumPy(m)
        mf_np_counts = mfgen.GetCountFingerprintAsNumPy(m)

        rd_np_bits = rdgen.GetFingerprintAsNumPy(m)
        rd_np_counts = rdgen.GetCountFingerprintAsNumPy(m)

        ap_np_bits = apgen.GetFingerprintAsNumPy(m)
        ap_np_counts = apgen.GetCountFingerprintAsNumPy(m)

        tt_np_bits = ttgen.GetFingerprintAsNumPy(m)
        tt_np_counts = ttgen.GetCountFingerprintAsNumPy(m)

        # print non zero value and posizion in the array
        #print(np.nonzero(mf_np_bits), mf_np_bits[np.nonzero(np_bits)])
        #print(np.nonzero(mf_np_counts), mf_np_counts[np.nonzero(np_counts)])

        if np.count_nonzero(mf_np_bits) > mf_maxnonzero:
            mf_maxnonzero = np.count_nonzero(mf_np_bits)
        if np.count_nonzero(mf_np_counts) > mf_maxnonzero:
            mf_maxnonzero = np.count_nonzero(mf_np_counts)

        if np.count_nonzero(rd_np_bits) > rd_maxnonzero:
            rd_maxnonzero = np.count_nonzero(rd_np_bits)
        if np.count_nonzero(rd_np_counts) > rd_maxnonzero:
            rd_maxnonzero = np.count_nonzero(rd_np_counts)

        if np.count_nonzero(ap_np_bits) > ap_maxnonzero:
            ap_maxnonzero = np.count_nonzero(ap_np_bits)
        if np.count_nonzero(ap_np_counts) > ap_maxnonzero:
            ap_maxnonzero = np.count_nonzero(ap_np_counts)

        if np.count_nonzero(tt_np_bits) > tt_maxnonzero:
            tt_maxnonzero = np.count_nonzero(tt_np_bits)
        if np.count_nonzero(tt_np_counts) > tt_maxnonzero:
            tt_maxnonzero = np.count_nonzero(tt_np_counts)                

    print(fingersize, "Max non zero value in FPs is: ", mf_maxnonzero, 
          rd_maxnonzero, ap_maxnonzero, tt_maxnonzero)


We have  151  molecules 
8 Max non zero value in FPs is:  6 7 5 4
16 Max non zero value in FPs is:  7 10 6 4
32 Max non zero value in FPs is:  7 13 6 4
64 Max non zero value in FPs is:  7 16 6 4
128 Max non zero value in FPs is:  7 17 6 4
256 Max non zero value in FPs is:  7 17 6 4
512 Max non zero value in FPs is:  7 18 6 4
1024 Max non zero value in FPs is:  7 18 6 4
2048 Max non zero value in FPs is:  7 18 6 4
4096 Max non zero value in FPs is:  8 18 6 4


Using VS3 descriptors: I need to work on that as we are dealing with small maolecules not all can be computed also considering the elements involved 

considering the previous I will use 512 but need to check similarity and maybe decrease 

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

for fingersize in [8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]:

    mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=fingersize)
    rdgen = rdFingerprintGenerator.GetRDKitFPGenerator(fpSize=fingersize)
    apgen = rdFingerprintGenerator.GetAtomPairGenerator(fpSize=fingersize)
    ttgen = rdFingerprintGenerator.GetTopologicalTorsionGenerator(fpSize=fingersize)

    mf_np_bits_l = []
    mf_np_counts_l = []
    rd_np_bits_l = []
    rd_np_counts_l = []
    ap_np_bits_l = []
    ap_np_counts_l = []
    tt_np_bits_l = []
    tt_np_counts_l = []

    for m in ms:
        mf_np_bits = mfgen.GetFingerprintAsNumPy(m)
        mf_np_bits_l.append(mf_np_bits)
        mf_np_counts = mfgen.GetCountFingerprintAsNumPy(m)
        mf_np_counts_l.append(mf_np_counts)

        rd_np_bits = rdgen.GetFingerprintAsNumPy(m)
        rd_np_bits_l.append(rd_np_bits)
        rd_np_counts = rdgen.GetCountFingerprintAsNumPy(m)
        rd_np_counts_l.append(rd_np_counts)

        ap_np_bits = apgen.GetFingerprintAsNumPy(m)
        ap_np_bits_l.append(ap_np_bits)
        ap_np_counts = apgen.GetCountFingerprintAsNumPy(m)
        ap_np_counts_l.append(ap_np_counts)

        tt_np_bits = ttgen.GetFingerprintAsNumPy(m)
        tt_np_bits_l.append(tt_np_bits)
        tt_np_counts = ttgen.GetCountFingerprintAsNumPy(m)
        tt_np_counts_l.append(tt_np_counts)

    mf_np_bits_l = np.array(mf_np_bits_l)
    mf_np_counts_l = np.array(mf_np_counts_l)
    rd_np_bits_l = np.array(rd_np_bits_l)
    rd_np_counts_l = np.array(rd_np_counts_l)
    ap_np_bits_l = np.array(ap_np_bits_l)
    ap_np_counts_l = np.array(ap_np_counts_l)
    tt_np_bits_l = np.array(tt_np_bits_l)
    tt_np_counts_l = np.array(tt_np_counts_l)

    mf_sim = cosine_similarity(mf_np_bits_l)
    rd_sim = cosine_similarity(rd_np_bits_l)
    ap_sim = cosine_similarity(ap_np_bits_l)
    tt_sim = cosine_similarity(tt_np_bits_l)

    mf_sim_counts = cosine_similarity(mf_np_counts_l)
    rd_sim_counts = cosine_similarity(rd_np_counts_l)
    ap_sim_counts = cosine_similarity(ap_np_counts_l)
    tt_sim_counts = cosine_similarity(tt_np_counts_l)

    print(fingersize, "Mean similarity in FPs is: ", np.average(mf_sim),
            np.average(rd_sim), np.average(ap_sim), np.average(tt_sim))
    print(fingersize, "Mean similarity in FPs is: ", np.average(mf_sim_counts),
            np.average(rd_sim_counts), np.average(ap_sim_counts), np.average(tt_sim_counts))

8 Mean similarity in FPs is:  0.34324721077245396 0.22935772953079098 0.2965502184173368 0.0017543090215341433
8 Mean similarity in FPs is:  0.31731350999440344 0.21203055848943778 0.11413508979340964 0.0010087276873821324
16 Mean similarity in FPs is:  0.1975690395664565 0.1286229005371921 0.1817531525249454 0.0011841585895355467
16 Mean similarity in FPs is:  0.1902418994036678 0.11903690826506591 0.06887275681279546 0.0007455813341520109
32 Mean similarity in FPs is:  0.13349737948985188 0.08399583437053926 0.10131994718844958 0.0008771545107670716
32 Mean similarity in FPs is:  0.13559229594737818 0.08253684040501515 0.04214535979438029 0.0004824349809218894
64 Mean similarity in FPs is:  0.09116340931409354 0.05166192396127244 0.060559228332079704 0.0007455813341520109
64 Mean similarity in FPs is:  0.09584193637276041 0.05169871879711153 0.027150065048888002 0.0004824349809218894
128 Mean similarity in FPs is:  0.07055939119290774 0.04124662459551662 0.03874509230649678 0.0004824

In [5]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d 

from sklearn.decomposition import PCA

fingersize = 16
fgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=fingersize)
fgen = rdFingerprintGenerator.GetRDKitFPGenerator(fpSize=fingersize)

np_bits_l = []
np_counts_l = []

for m in ms:
    np_bits = fgen.GetFingerprintAsNumPy(m)
    np_bits_l.append(np_bits)
    np_counts = fgen.GetCountFingerprintAsNumPy(m)
    np_counts_l.append(np_counts)

pca = PCA(n_components=3)
pca.fit(np_bits_l)
V = pca.components_
print(pca.explained_variance_ratio_)
print(V.shape)
print(V[0,:])


[0.15584709 0.11228172 0.10229228]
(3, 16)
[0.23130913 0.04424128 0.17101353 0.02300183 0.07151571 0.30243379
 0.16598249 0.15713942 0.2506692  0.09480811 0.17598736 0.56103528
 0.1354981  0.17607936 0.42685025 0.34178917]


testa some basic models

In [9]:
fp = open('./data/labels.txt', 'r')

lnames = []
labels = []
for line in fp:
    sline = line.strip().split()
    name = sline[1]
    lnames.append(name)
    labels.append(float(sline[-1]))

fingersize = 16
fgen = rdFingerprintGenerator.GetRDKitFPGenerator(fpSize=fingersize)

np_bits_l = []
np_counts_l = []

for m in ms:
    np_bits = fgen.GetFingerprintAsNumPy(m)
    np_bits_l.append(np_bits)
    np_counts = fgen.GetCountFingerprintAsNumPy(m)
    np_counts_l.append(np_counts)

np_bits_l = np.array(np_bits_l)
np_counts_l = np.array(np_counts_l)

fp.close()
print(len(lnames))
print(len(names))

140
152


In [7]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import (LeaveOneOut, cross_val_predict,
                                     cross_val_score, train_test_split)



